In [1]:
# @title Step 1: Authenticate with Google
# @markdown Note: You will be asked to sign in with Google, connected to your Lamini account.

from google.colab import auth
import requests
import os
import yaml

def authenticate_powerml():
  auth.authenticate_user()
  gcloud_token = !gcloud auth print-access-token
  powerml_token_response = requests.get('https://api.powerml.co/v1/auth/verify_gcloud_token?token=' + gcloud_token[0])
  print(powerml_token_response)
  return powerml_token_response.json()['token']

key = authenticate_powerml()

config = {
    "production": {
        "key": key,
        "url": "https://api.powerml.co"
    }
}

keys_dir_path = '/root/.powerml'
os.makedirs(keys_dir_path, exist_ok=True)

keys_file_path = keys_dir_path + '/configure_llama.yaml'
with open(keys_file_path, 'w') as f:
  yaml.dump(config, f, default_flow_style=False)

<Response [200]>


In [2]:
# @title Step 2: Install the open-source [Lamini library](https://pypi.org/project/lamini/) to use LLMs easily
# @markdown Note: After installing, click the "RESTART RUNTIME" button at the end of the output, then go onto the next cell.
# @markdown Lamini is just on a more recent version of numpy than Colab.
!pip install --upgrade --force-reinstall --ignore-installed lamini

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 89.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 kB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.5/502.5 kB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
from llama import Type, Context

class Input(Type):
    question: str = Context("question")

class Output(Type):
    answer: str = Context("answer to question")

In [2]:
from llama import InputOutputRunner

llm = InputOutputRunner(input_type=Input, output_type=Output, model_name="EleutherAI/pythia-410m")

In [4]:
!wget -q -O "lamini_docs.jsonl" "https://drive.google.com/uc?export=download&id=1rJDDI2wvEL4npvtOUaJ_-1zuCjBgSxHw"

In [ ]:
llm.load_data_from_jsonlines("fine_tune_data.jsonl", verbose=True)

In [11]:
# test llm before training
# new_input = Input(question="How do I add data? Please help")
# llm(new_input)
# print(llm(new_input))

new_input = Input(question="What is 7 plus 35? Please help")
llm(new_input)

Output(answer='A:\n\nYou can use the following code:\nimport re\n\ndef answer(question):\n    return re.search(r\'[0-9]+\', question).group(1)\n\ndef question(question):\n    return re.search(r\'[0-9]+\', question).group(1)\n\ndef generate(question):\n    answer = answer(question)\n    return answer\n\ndef main():\n    question = "What is 7 plus 35?"\n    answer = "7+35"\n    answer = answer(question)\n    print(answer)\n\nif __name__ == \'__main__\':\n    main()\n\nOutput:\n7+35\n\nA:\n\nYou can use the following:\nimport re\n\ndef answer(question):\n    return re.search(r\'[0-9]+\', question).group(1)\n\ndef question(question):\n    return re.search(r\'[0-9]+\', question).group(1)\n\ndef generate(question):\n    answer = answer(question)\n    return answer\n\ndef main():\n    question = "What is')

In [17]:
llm.train(is_public=True)

Training job submitted! Check status of job 3354 here: https://api.powerml.co/train/3354
Finetuning process completed, model name is: 5db2c8b95417b1d697dacec86401aa1868fe62540af18da63e14d4ce01599bec


In [18]:
llm.evaluate()

{'job_id': 3354,
 'eval_results': [{'input': 'question (question): Does the documentation have a secret code that unlocks a hidden treasure?',
   'outputs': [{'model_name': '5db2c8b95417b1d697dacec86401aa1868fe62540af18da63e14d4ce01599bec',
     'output': 'answer:  The documentation has a secret code that unlocks a hidden treasure. You can find it in the "secret_code" field in the "answer:" field.'},
    {'model_name': 'Base model (EleutherAI/pythia-410m)',
     'output': 'answer: \n\nA:\n\nThe answer to your question is "yes".\nThe answer to your question is "no".\nThe answer to your question is "yes".\nThe answer to your question is "no".\nThe answer to your question is "yes".\nThe answer to your question is "no".\nThe answer to your question is "yes".\nThe answer to your question is "no".\nThe answer to your question is "yes".\nThe answer to your question is "no".\nThe answer to your question is "yes".\nThe answer to your question is "no".\nThe answer to your question is "yes".\nThe

In [19]:
# inference
new_input = Input(question="Your witness counselor.")
llm(new_input)

Output(answer='Question:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:\nQuestion:')

In [20]:
new_input = Input(question="Who the hell are you?")
llm(new_input)

Output(answer='Question:')

In [ ]:
# using the model later
#model_name = "5db2c8b95417b1d697dacec86401aa1868fe62540af18da63e14d4ce01599bec" # Or your model ID here

In [ ]:
#llm_later = InputOutputRunner(input_type=Input, output_type=Output, model_name=model_name)